In [2]:
a = 8479994658316772151941616510097127087554541274812435112009425778595495359700244470400642403747058566807127814165396640215844192327900454116257979487432016769329970767046735091249898678088061634796559556704959846424131820416048436501387617211770124292793308079214153179977624440438616958575058361193975686620046439877308339989295604537867493683872778843921771307305602776398786978353866231661453376056771972069776398999013769588936194859344941268223184197231368887060609212875507518936172060702209557124430477137421847130682601666968691651447236917018634902407704797328509461854842432015009878011354022108661461024768
p = 30531851861994333252675935111487950694414332763909083514133769861350960895076504687261369815735742549428789138300843082086550059082835141454526618160634109969195486322015775943030060449557090064811940139431735209185996454739163555910726493597222646855506445602953689527405362207926990442391705014604777038685880527537489845359101552442292804398472642356609304810680731556542002301547846635101455995732584071355903010856718680732337369128498655255277003643669031694516851390505923416710601212618443109844041514942401969629158975457079026906304328749039997262960301209158175920051890620947063936347307238412281568760161

In [1]:
import random
import logging
import sys
from cryptolib import invmod as inverse_modulo, is_prime

_logger = logging.getLogger("tonellishanks")


def legendre_symbol(a: int, p: int, /) -> int:

    assert p % 2 != 0

    return pow(a, (p - 1) >> 1, p)


def _choose_b(p: int, /, *, det=True) -> int:

    assert p > 2
    assert p % 2 != 0

    b = 2
    _attempts = 1

    if det:
        while legendre_symbol(b, p) == 1:
            b += 1
            _attempts += 1
    else:
        while legendre_symbol(b, p) == 1:
            b = random.randrange(2, p)
            _attempts += 1

    assert b < p
    assert legendre_symbol(b, p) == p - 1

    _logger.info("Found b = %d after %d attempts", b, _attempts)

    return b


def _tonelli_shanks_recursive(a: int, k: int, p: int, b: int, b_inverse: int, /):
    """
    Computes a square root of a modulo prime p
    :param a: the number to take the square root of
    :param k: positive integer, such that a^m = 1 (mod p) where m = (p-1)/(2^k)
    :param p: odd prime p modulo which we are working
    :param b: an arbitrary non-square modulo p
    :param b_inverse: the inverse of b modulo p, i.e., b * b_inverse = 1 (mod p)
    :return: one of the square roots of a modulo p (the other can be obtained via negation modulo p)
    """

    assert p > 2
    assert 0 < a < p
    assert k > 0

    m = (p - 1) >> k

    # assumption
    assert pow(a, m, p) == 1

    a_m = 1

    # check that b is indeed a non-square modulo p
    assert legendre_symbol(b, p) == p - 1

    _logger.info("-------- [New round] --------")
    _logger.info("a = %d, m = %d, a^m = 1", a, m)

    while m % 2 == 0 and a_m == 1:

        m >>= 1
        k += 1

        assert m == (p - 1) >> k

        a_m = pow(a, m, p)

        _logger.info(
            "m is even and a^m = 1 => we divide m by 2 and get: m = %d, a^m = %s",
            m,
            "1" if a_m == 1 else "-1"
        )

        # since Z/pZ is a field, there cannot be any roots for 1 apart from 1 and -1
        assert a_m == 1 or a_m == p - 1

    assert a_m == 1 or a_m == p - 1

    if a_m == p - 1:
        # a^m = -1 (mod p)
        _logger.info("m = %d, a^m = -1 => we multiply a^m with a legendre symbol of a non-square b modulo p", m)
        assert k >= 2
        b_power = 1 << (k - 1)
        b_power_half = 1 << (k - 2)
        assert pow(a, m, p) == p - 1
        assert b_power * m == (p - 1) >> 1
        a_next = (a * pow(b, b_power, p)) % p
        _logger.info("(a * b^%d)^m = (a * b^%d)^%d = %d^%d = 1", b_power, b_power, m, a_next, m)
        _logger.info(
            "It follows that a_next := a * b^%d = %d * %d = %d is a square whose root yields a root of a",
            b_power,
            a,
            pow(b, b_power, p),
            a_next
        )
        assert pow(a_next, m, p) == 1
        a_next_root = _tonelli_shanks_recursive(a_next, k, p, b, b_inverse)
        _logger.info("The root of a_next = %d is %d", a_next, a_next_root)
        a_root = a_next_root * pow(b_inverse, b_power_half, p)
        _logger.info("sqrt(a_next)^2 = %d^2 = a_next = a * b^%d = sqrt(a)^2 * b^%d", a_next_root, b_power, b_power)
        _logger.info(
            "=> sqrt(a = %d) = sqrt(a_next) * b^(-%d) = %d * %d = %d",
            a,
            b_power_half,
            a_next_root,
            pow(b_inverse, b_power_half, p),
            a_root
        )
        _logger.info("-------- [Round complete] --------")
        return a_root % p
    assert a_m == 1
    assert m % 2 == 1
    _logger.info("-------- [Round complete] --------")
    # we now handle the case when m is odd
    # this case is easy, a^((m+1)/2) is a square root of a
    return pow(a, (m + 1) >> 1, p)


def tonelli_shanks(a: int, p: int, /, *, deterministic=True) -> int | None:
    """
    Computes a square root of a modulo prime p
    :param a: the number to take the square root of
    :param p: odd prime p modulo which we are working
    :param deterministic: whether to search for the non-square b deterministically
    :return: one of the square roots of a modulo p (the other can be obtained via negation modulo p)
    """
    assert p > 2
    assert 0 < a < p
    # quick Fermat primality test
    assert pow(a, p - 1, p) == 1
    if legendre_symbol(a, p) != 1:
        # a is not not a square modulo p
        return None
    _logger.info("======== [Starting algorithm with a = %d, p = %d] ========", a, p)
    b = _choose_b(p, det=deterministic)
    b_inverse = inverse_modulo(b, p)
    assert b * b_inverse % p == 1
    return _tonelli_shanks_recursive(a, 1, p, b, b_inverse)

In [4]:
r = tonelli_shanks(a, p)
print(r)
print(pow(r, 2, p))

2362339307683048638327773298580489298932137505520500388338271052053734747862351779647314176817953359071871560041125289919247146074907151612762640868199621186559522068338032600991311882224016021222672243139362180461232646732465848840425458257930887856583379600967761738596782877851318489355679822813155123045705285112099448146426755110160002515592418850432103641815811071548456284263507805589445073657565381850521367969675699760755310784623577076440037747681760302434924932113640061738777601194622244192758024180853916244427254065441962557282572849162772740798989647948645207349737457445440405057156897508368531939120
84799946583167721519416165100971270875545412748124351120094257785954953597002444704006424037470585668071278141653966402158441923279004541162579794874320167693299707670467350912498986780880616347965595567049598464241318204160484365013876172117701242927933080792141531799776244404386169585750583611939756866200464398773083399892956045378674936838727788439217713073056027763987869783538